In [1]:
import numpy as np
import matplotlib.pyplot as plt
import time
import random

import numpy as np
import matplotlib.pyplot as plt
import torch
from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import TensorDataset, DataLoader
from torch.autograd import Variable
import pyDOE

from torch.optim.optimizer import Optimizer, required

torch.manual_seed(16)
np.random.seed(16)
#random.seed(16)
torch.backends.cudnn.deterministic = True

In [3]:
import scipy.io
mat = scipy.io.loadmat(r'C:\Users\krist\Downloads\cylinder_nektar_wake.mat')

In [3]:
u = mat['U_star']
p = mat['p_star']
x = mat['X_star']
t = mat['t']

In [4]:
#create data
residual = []
boundary = []
for i in range(70):
    data = list(zip(x,u[:,:,i],p[:,i]))
    pairs = random.sample(data,2000)
    residual += [list(np.concatenate((j[0],t[i],j[1],[j[2]]))) for j in pairs]
    
    y_bound = data[:100] + data[-100:]
    pairs_y = random.sample(y_bound,100)
    x_bound = data[::100] + data[99::100]
    pairs_x = random.sample(x_bound,50)
    
    boundary += [list(np.concatenate((j[0],t[i],j[1],[j[2]]))) for j in pairs_x]
    boundary += [list(np.concatenate((j[0],t[i],j[1],[j[2]]))) for j in pairs_y]

    
residual = np.array(residual)
initial = residual[:2000]
boundary = np.array(boundary)
    

In [5]:
#to tensor
def to_tensor(a):
    a = a.reshape((-1,1))
    a = torch.tensor(a).type(torch.FloatTensor)
    return a
        
        
#residual
x_r = to_tensor(residual[:,0])
y_r = to_tensor(residual[:,1])
t_r = to_tensor(residual[:,2])
u_r = to_tensor(residual[:,3])
v_r = to_tensor(residual[:,4])
p_r = to_tensor(residual[:,5])

#boundary
x_b = to_tensor(boundary[:,0])
y_b = to_tensor(boundary[:,1])
t_b = to_tensor(boundary[:,2])
u_b = to_tensor(boundary[:,3])
v_b = to_tensor(boundary[:,4])
p_b = to_tensor(boundary[:,5])

#initial
x_i = to_tensor(initial[:,0])
y_i = to_tensor(initial[:,1])
t_i = to_tensor(initial[:,2])
u_i = to_tensor(initial[:,3])
v_i = to_tensor(initial[:,4])
p_i = to_tensor(initial[:,5])

In [6]:
class PINN(nn.Module):
    def __init__(self,hidden_number,hidden_size):
        super(PINN, self).__init__()
        self.inLayer = nn.Linear(3, hidden_size)
        self.linears = nn.ModuleList([nn.Linear(hidden_size, hidden_size) for i in range(hidden_number)])
        self.outLayer = nn.Linear(hidden_size, 3)

    def forward(self, x1, x2, x3):
        x  = torch.cat((x1,x2,x3),1)
        x = self.inLayer(x)
        for i, layer in enumerate(self.linears):
            x = torch.tanh(x)
            x = layer(x)
        x = torch.tanh(x)  
        x = self.outLayer(x)
        return x

In [7]:
pn = PINN(10,100)

MAX_EPOCHS = 10000
LRATE = 1e-4

#Use Adam for training
optimizer = torch.optim.Adam(pn.parameters(), lr=LRATE)

criterion = nn.MSELoss()

loss_history_u = []
loss_history_f = []
loss_history = []

In [9]:
l=1
for epoch in range(10):
    #full batch
    
    #bc
    x_bc = x_b.clone()
    x_bc.requires_grad = True

    y_bc = y_b.clone()
    y_bc.requires_grad = True
    
    t_bc = t_b.clone()
    t_bc.requires_grad = True
    
    upred_bc = pn(x_bc, y_bc, t_bc)
    
    u_bc = torch.reshape(upred_bc[:,0], (-1, 1))
    v_bc = torch.reshape(upred_bc[:,1], (-1, 1))

    loss_bc = criterion(input=u_bc, target = u_b) + criterion(input=v_bc, target = v_b)

    #loss_history_u.append([epoch, loss_bc])
    
    #ic
    x_ic = x_i.clone()
    x_ic.requires_grad = True

    y_ic = y_i.clone()
    y_ic.requires_grad = True
    
    t_ic = t_i.clone()
    t_ic.requires_grad = True
    
    upred_ic = pn(x_ic, y_ic, t_ic)
    
    u_ic = torch.reshape(upred_ic[:,0], (-1, 1))
    v_ic = torch.reshape(upred_ic[:,1], (-1, 1))

    loss_ic = criterion(input=u_ic, target = u_i) + criterion(input=v_ic, target = v_i)

    #loss_history_u.append([epoch, loss_ic])

    #residual
    x_res = x_r.clone()
    x_res.requires_grad = True

    y_res = y_r.clone()
    y_res.requires_grad = True
    
    t_res = t_r.clone()
    t_res.requires_grad = True

    upred = pn(x_res, y_res, t_res)
    
    u = torch.reshape(upred[:,0], (-1, 1))
    v = torch.reshape(upred[:,1], (-1, 1))
    p = torch.reshape(upred[:,2], (-1, 1))
    
    u_t = torch.autograd.grad(u.sum(),t_res,create_graph=True)[0]
    v_t = torch.autograd.grad(v.sum(),t_res,create_graph=True)[0]
    
    u_x = torch.autograd.grad(u.sum(),x_res,create_graph=True)[0]
    v_x = torch.autograd.grad(v.sum(),x_res,create_graph=True)[0]
    
    u_y = torch.autograd.grad(u.sum(),y_res,create_graph=True)[0]
    v_y = torch.autograd.grad(v.sum(),y_res,create_graph=True)[0]
    
    p_x = torch.autograd.grad(p.sum(),x_res,create_graph=True)[0]
    p_y = torch.autograd.grad(p.sum(),y_res,create_graph=True)[0]

    u_xx = torch.autograd.grad(u_x.sum(),x_res,create_graph=True)[0]
    u_yy = torch.autograd.grad(u_y.sum(),y_res,create_graph=True)[0]
    
    v_xx = torch.autograd.grad(v_x.sum(),x_res,create_graph=True)[0]
    v_yy = torch.autograd.grad(v_y.sum(),y_res,create_graph=True)[0]


    loss_r = criterion(input = u_t + u*u_x + v*u_y + p_x - (1/100)*(u_xx + u_yy), target = 0 )\
          + criterion(input = v_t + u*v_x + v*v_y + p_y - (1/100)*(v_xx + v_yy), target = 0 )\
          + criterion(input = u_x + v_y, target = 0 )
    

    loss_history_f.append([epoch, loss_r])
    
    
    loss = loss_bc + loss_ic + loss_r
    loss_history.append([epoch, loss])
    
    #optimizer step
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
        
        
    if (epoch+1) % 1 == 0:
        print("Epoch: {}, MSE_u: {:.7f}, MSE_f: {:.7f}, MSE: {:.7f}, l: {:.2f}".format((epoch+1), mse_u, mse_f, loss, l))

KeyboardInterrupt: 

In [52]:
mat['X_star'][:,0].reshape(50,100)

array([[1.        , 1.07070707, 1.14141414, ..., 7.85858586, 7.92929293,
        8.        ],
       [1.        , 1.07070707, 1.14141414, ..., 7.85858586, 7.92929293,
        8.        ],
       [1.        , 1.07070707, 1.14141414, ..., 7.85858586, 7.92929293,
        8.        ],
       ...,
       [1.        , 1.07070707, 1.14141414, ..., 7.85858586, 7.92929293,
        8.        ],
       [1.        , 1.07070707, 1.14141414, ..., 7.85858586, 7.92929293,
        8.        ],
       [1.        , 1.07070707, 1.14141414, ..., 7.85858586, 7.92929293,
        8.        ]])

In [5]:
mat['U_star'][:,0,0]

array([1.11419216, 1.11102707, 1.10748452, ..., 1.04183122, 1.03144583,
       1.02128975])

In [18]:
x = mat['X_star'][:,0].reshape(50,100)[::2,::2]
y = mat['X_star'][:,1].reshape(50,100)[::2,::2]


for i in range(20):
    u = mat['U_star'][:,0,i].reshape(50,100)[::2,::2]
    v = mat['U_star'][:,1,i].reshape(50,100)[::2,::2]

    plt.figure(figsize=(20,10))

    m = np.hypot(u, u)
    plt.title(label = f'time: {i/10} sec')
    plt.quiver(x,y,u,v,m)
    

    plt.savefig(f'C:\\Users\\krist\\OneDrive\\Υπολογιστής\\figures2\\{i}.png')
    plt.close()

In [44]:
%matplotlib qt

In [54]:
fig = plt.figure()
ax = plt.axes(projection='3d')
ax.plot_wireframe(mat['X_star'][:,0].reshape(50,100), mat['X_star'][:,1].reshape(50,100), mat['U_star'][:,0,40].reshape(50,100), color='r')